In [1]:
import warnings
warnings.simplefilter("ignore")
from sys import exit
import os
import cv2
import numpy as np
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import askdirectory
from scipy import ndimage as ndi
from skimage import measure, io, filters, morphology, util, restoration, color
from pathlib import Path
from skimage.segmentation import watershed, random_walker
from skimage.feature import peak_local_max
import matplotlib.pyplot as plt
from math import sqrt
import csv
import copy
import scipy.optimize as opt

In [2]:
'''
showimgs: shows all images for testing, set as false for just running
saveimgs: allows user to save images
'''
showimgs = False
saveimgs = True

In [3]:
def adaptiveFlatten(flist):
    iterables = ["<class 'tuple'>","<class 'list'>","<class 'numpy.ndarray'>"]
    alist = flist.copy()
    while True:
        blist = []
        br = True
        for tit in alist:
            if str(type(tit)) in iterables:
                if len(tit) >1:
                    br = False
                    break
        if br == True:
            break
        for eone in alist:
            if str(type(eone)) in iterables:
                for item in list(eone):
                    blist.append(item)
            else:
                blist.append(eone)
        alist = blist.copy()
    return alist

In [4]:
'''Ask for picture location'''
print("\n please choose video folder!")
dirtouse = askdirectory(title = "Select foldername", initialdir = '/home/edwin/Desktop/')
print(dirtouse)


tifs = []
imgs = []

'''Move through all files in directory, saving file names and images if they're tifs'''
for filename in sorted(os.listdir(dirtouse)):
    if filename.endswith('.tif'):
        tifs.append(filename)
        pictouse = os.path.join(dirtouse,filename)
        img = io.imread(pictouse,-1)
        imgs.append(img)

length = len(imgs)
print(length)


 please choose video folder!
/home/ewinden/Desktop/Default
200


In [11]:
colors = np.array([[0,0,0],[255, 0, 0], [0, 255, 0],[0, 0, 255],[255,255,0],[0,255,255],[255,0,255],
                  [125,0,0],[0,125,0],[0,0,125],[125,125,0],[125,0,125],[0,125,125],[125,125,125]])
footprint = morphology.disk(15)

avm = imgs[-1].copy()
for im in imgs[-5:-2]:
    avm = np.minimum(im,avm)
wtha = morphology.white_tophat(avm, footprint)
otsua = wtha > filters.threshold_otsu(wtha)
lotsua,k = measure.label(otsua, return_num=True)
sizes = np.bincount(lotsua.ravel())
mask_sizes = sizes > 4
mask_sizes[0] = 0
otsua_cleaned = mask_sizes[lotsua]
lotsua,k = measure.label(otsua_cleaned, return_num=True)
peaks = peak_local_max(filters.gaussian(avm,preserve_range=True),min_distance=5, num_peaks=k)
markers = np.zeros_like(avm)
markers[tuple(peaks.T)] = 65535
label, _ = ndi.label(markers)
if saveimgs == True:
    spath = os.path.join(dirtouse,"setupimgs")
    ppath = os.path.join(dirtouse,"processedimgs")
    os.makedirs(spath,exist_ok=True)
    os.makedirs(ppath,exist_ok=True)
    io.imsave(os.path.join(spath,"minimum.tif"),avm)
    io.imsave(os.path.join(spath,"otsu.tif"),otsua_cleaned)
    io.imsave(os.path.join(spath,"markers.tif"),markers)


imcentroids = []
for imgn in range(length-1,-1,-1):
    print(imgn)
    img = filters.gaussian(imgs[imgn],preserve_range=True)
    tophat = morphology.white_tophat(img, footprint)
    otsu = tophat > filters.threshold_otsu(tophat)
    otsul, _ = ndi.label(otsu)
    sizes = np.bincount(otsul.ravel())
    mask_sizes = sizes > 4
    mask_sizes[0] = 0
    otsu_cleaned = mask_sizes[otsul]
    w = watershed(-img, label, mask=otsu_cleaned, watershed_line=True)
    pl = []
    imcentroids.append([])
    for region in measure.regionprops(w, intensity_image=img):
        imcentroids[-1].append(adaptiveFlatten([imgn,[region.label-1,region.centroid_weighted,
                                region.axis_major_length,region.axis_minor_length,np.sum(region.image_intensity),
                                region.moments_hu, region.equivalent_diameter_area,region.area]]))
        pl.append([int(region.centroid_weighted[0]),int(region.centroid_weighted[1])])
    wbg = util.invert(w < 1)
    flatw = (np.remainder(w,12)+1) * wbg
    c = util.img_as_ubyte(colors[flatw])
    if saveimgs == True:
        io.imsave(os.path.join(ppath,str(imgn)+"_w.tif"),w)
        io.imsave(os.path.join(ppath,str(imgn)+"_c.jpeg"),c)

In [14]:
loc = os.path.join(os.path.join(dirtouse,"backTracked.csv"))
with open(loc, "w") as f:
    write = csv.writer(f)
    write.writerow(["ImageNumber","Label","CentroidY","CentroidX","MajorAxis","MinorAxis",
                        "TotalIntensity","M1","M2","M3","M4","M5","M6","M7","Diameter","Area"])
    for fg in imcentroids:
        write.writerows(fg)

bonus = []
nlabels = 0
for s in imcentroids:
    for l in s:
        if l[1]>=nlabels:
            nlabels = l[1]+1
            
nframes = len(imcentroids)

data = np.zeros((nframes+1,nlabels+1))

for i in range(nframes+1):
    data[i][0]=i-1
for i in range(nlabels+1):
    data[0][i]=i-1

for s in imcentroids:
    for l in s:
        imgn,lab,inten = l[0],l[1],l[6]/l[15]
        data[imgn+1][lab+1]=inten

loc = os.path.join(os.path.join(dirtouse,"forGraphs.csv"))
with open(loc, "w") as f:
    write = csv.writer(f)
    for fg in data:
        write.writerow(fg)